In [1]:
import pandas as pd
import numpy as np 
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers, Model
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

2024-04-24 15:56:33.985306: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Create TableGan Model 

In [ ]:
df = pd.read_csv("com_salary.csv")

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
selected_df= df[['ADP', 'Avg Salary', 'Median Salary', 'Min Salary', 'Max Salary', 'rPTS', 'PTS', 'Dollars']]


In [ ]:
selected_df.head()

## Step 1: Define the Generator
The Generator's role in a GAN is to create synthetic data that is indistinguishable from real data. It learns to do this through the adversarial process with the Discriminator. Here, we define a simple neural network model for the


# Define the Generator
class Generator(Model):
    def __init__(self, z_dim):
        super(Generator, self).__init__()
        self.dense1 = layers.Dense(64, activation='relu')
        self.out = layers.Dense(z_dim, activation='tanh')

    def call(self, x):
        x = self.dense1(x)
        return self.out(x)



## Step 2: Define the Discriminator
The Discriminator acts as a classifier that tries to distinguish real data from fake data produced by the Generator. This class is also defined with a simple architecture, consisting of one hidden layer.


In [ ]:
# Define the Discriminator
class Discriminator(Model):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.dense1 = layers.Dense(64, activation='relu')
        self.out = layers.Dense(1, activation='sigmoid')

    def call(self, x):
        x = self.dense1(x)
        return self.out(x)

## Step 3: Define the TableGAN Model
Here we integrate the Generator and Discriminator into a complete GAN model. The TableGAN class manages the training loop where both models are trained in an adversarial setup. This structure includes methods for compiling the model and defining the training step, utilizing TensorFlow's capabilities.


In [ ]:
# Define the TableGAN model
class TableGAN(Model):
    def __init__(self, input_dim, z_dim):
        super(TableGAN, self).__init__()
        self.generator = Generator(z_dim)
        self.discriminator = Discriminator(input_dim)

    def compile(self, g_optimizer, d_optimizer, loss_function):
        super(TableGAN, self).compile()
        self.g_optimizer = g_optimizer
        self.d_optimizer = d_optimizer
        self.loss_function = loss_function

    def train_step(self, real_data):
        batch_size = tf.shape(real_data)[0]
        z_dim = self.generator.layers[-1].output_shape[-1]

        random_latent_vectors = tf.random.normal(shape=(batch_size, z_dim))

        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            generated_data = self.generator(random_latent_vectors)
            real_output = self.discriminator(real_data)
            fake_output = self.discriminator(generated_data)

            gen_loss = self.loss_function(tf.ones_like(fake_output), fake_output)
            real_loss = self.loss_function(tf.ones_like(real_output), real_output)
            fake_loss = self.loss_function(tf.zeros_like(fake_output), fake_output)
            disc_loss = real_loss + fake_loss

        gradients_of_generator = gen_tape.gradient(gen_loss, self.generator.trainable_variables)
        gradients_of_discriminator = disc_tape.gradient(disc_loss, self.discriminator.trainable_variables)

        self.g_optimizer.apply_gradients(zip(gradients_of_generator, self.generator.trainable_variables))
        self.d_optimizer.apply_gradients(zip(gradients_of_discriminator, self.discriminator.trainable_variables))

        return {'gen_loss': gen_loss, 'disc_loss': disc_loss}

## Step 4: Data Loading and Preprocessing
Before training the model, it's necessary to load and preprocess your data. This step involves reading a CSV file, detecting numerical and categorical columns, and applying appropriate transformations such as scaling for numerical data and one-hot encoding for categorical data.


In [ ]:
# Load and preprocess the data
data = pd.read_csv('path_to_your_dataset.csv')
numerical_cols = data.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = data.select_dtypes(include=['object', 'category']).columns.tolist()
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(), categorical_cols)
    ])
processed_data = preprocessor.fit_transform(data)


)



## Step 5: Initialize and Train the TableGAN Model
Finally, initialize the TableGAN model with dimensions derived from the preprocessed data, compile it with chosen optimizers and loss function, and train the model. Here, we specify the learning rates, the loss function, and the training parameters such as the number of epochs and batch size.


In [1]:
# Initialize the model and compile
input_dim = processed_data.shape[1]
z_dim = 50
tablegan = TableGAN(input_dim=input_dim, z_dim=z_dim)
tablegan.compile(
    g_optimizer=Adam(1e-4),
    d_optimizer=Adam(1e-4),
    loss_function=BinaryCrossentropy(from_logits=True)


# Train the model
epochs = 20
batch_size = 16
tablegan.fit(processed_data, epochs=epochs, batch_size=batch_size)

SyntaxError: incomplete input (2227514722.py, line 8)